## Functional Programming SIG 10.27.22

In [42]:
:opt no-lint
import Data.List
import GHC.Show
import Data.Char

### Algebraic Data Types Review

In [10]:
giveMeAValue :: Int -> Either Double Bool
giveMeAValue 0 = Right False
giveMeAValue _ = Left 10.01

In [11]:
parse :: String -> Either String [String]
parse s 
    | length s > 20 = Left "Error! cannot parse strings longer than 10 characters"
    | null s = Left "Error! empty string doesn't count okay"
    | otherwise = Right (words s)

In [12]:
spicyFunc :: (a -> b, a) -> b
spicyFunc (f, x) = f x 

In [13]:
(True, True) :: (Bool, Bool)
(True, False) :: (Bool, Bool)
(False, True) :: (Bool, Bool)
(False, False) :: (Bool, Bool)

(True,True)

(True,False)

(False,True)

(False,False)

### The data and type keywords

In [14]:
data MySumType = First | Second -- ~= Either () () ~= Bool

data MyProductType = Product Bool String -- ~= (Bool, String)

data JustADouble = ThisIsMyDouble Double -- ~= (Double)

foo :: JustADouble
foo = ThisIsMyDouble 89.9

In [15]:
data PizzaTopping = Pepperoni | Cheese | Olives | Pudding 
    deriving (Show)

favoriteTopping :: PizzaTopping
favoriteTopping = Olives

print favoriteTopping

Olives

In [16]:
data Shape = Circle (Double, Double) Double | Triangle (Double, Double) (Double, Double) (Double, Double)

In [17]:
type Point = (Double, Double)
type Radius = Double
data Shape = Circle Point Radius | Triangle Point Point Point
    deriving Show

In [18]:
gimmeAShape :: Bool -> Shape
gimmeAShape True = Circle (10.3,10.5) 100.0
gimmeAShape False = Triangle (0.0, 1.1) (1.1, 2.2) (2.2, 3.3)

gimmeAShape True

Circle (10.3,10.5) 100.0

In [19]:
eatShape :: Shape -> Double
eatShape (Circle center radius) = radius
eatShape (Triangle _ (_,y) _) = y

eatShape (gimmeAShape False)

2.2

In [20]:
data Boo = Tru | Fals

convert :: Boo -> Bool
convert Tru = True
convert Fals = False

#### Maybe

In [21]:
data Maybe a = Just a | Nothing 
    deriving (Show)

divide :: Int -> Int -> Int
divide x y = x `div` y

divide 10 5
divide 10 0

2

: 

In [22]:
divide' :: Int -> Int -> Maybe Int
divide' x 0 = Nothing
divide' x y = Just (x `div` y)

divide' 10 5
divide' 10 0

Just 2

Nothing

In [23]:
lameDivide :: Int -> Int -> Either String Int
lameDivide _ 0 = Left "no"
lameDivide 7 _ = Left "sorry I don't like 7"
lameDivide x y = Right (x `div` y)

lameDivide 9071 0
lameDivide 80 2

Left "no"

Right 40

In [24]:
:type Just
:type Nothing
:type Left
:type True
:type Pepperoni

Just :: forall a. a -> Maybe a

Nothing :: forall a. Maybe a

Left :: forall a b. a -> Either a b

True :: Bool

Pepperoni :: PizzaTopping

In [25]:
data Person = Person {name :: String, phone :: String, likesHaskell :: Bool}
    deriving Show
            
isaac :: Person
isaac = Person {name = "Isaac", phone = "12", likesHaskell = True}

phone isaac

"12"

In [26]:
:type name
:type phone
:type likesHaskell

name :: Person -> String

phone :: Person -> String

likesHaskell :: Person -> Bool

In [27]:
data Tree a = Empty | Node a (Tree a) (Tree a)
    deriving Show
    
myTree = Empty
myBetterTree = Node "I'm the root" Empty (Node "I'm a right child" Empty Empty)

In [28]:
insert :: Ord a => Tree a -> a -> Tree a
insert Empty val = Node val Empty Empty
insert (Node x left right) val = if val < x then Node x (insert left val) right else Node x left (insert right val)

insert (insert (insert Empty 10) 20) (-10)

Node 10 (Node (-10) Empty Empty) (Node 20 Empty Empty)

In [29]:
fromList :: Ord a => [a] -> Tree a
fromList xs = foldr (\x acc -> insert acc x) Empty xs

fromList [1,2,3,4,900,-98,0]

Node 0 (Node (-98) Empty Empty) (Node 900 (Node 4 (Node 3 (Node 2 (Node 1 Empty Empty) Empty) Empty) Empty) Empty)

In [30]:
toList :: Tree a -> [a]
toList Empty = []
toList (Node x left right) = toList left ++ [x] ++ toList right

In [31]:
getValue :: Tree a -> Maybe a 
getValue Empty = Nothing
getValue (Node x _ _) = Just x

In [32]:
getSubtrees :: Tree a -> [Tree a]
getSubtrees Empty = [Empty, Empty]
getSubtrees (Node x left right) = [left, right]

In [33]:
-- toString :: Show a => Tree a -> String
-- toString Empty = ""
-- toString tree = f [getValue tree] (getSubtrees tree)
--     where
--         f [] _ = ""
--         f vals trees = unwords (map (\x -> show' x ++ "  ") vals) ++ "\n" ++ f (map getValue trees) (concat (map getSubtrees trees))
--         show' Nothing = ""
--         show' (Just x) = show x
        
-- (putStrLn . toString . fromList) [1,2,3,4,900,-98,0]

### Function Composition and Point Free Style

In [34]:
(.) :: (b -> c) -> (a -> b) -> (a -> c)
f . g = \x -> f (g x)

In [40]:
toList (fromList [8,2,4,1,6,5,7,3])
(toList . fromList) [8,2,4,1,6,5,7,3]

[1,2,3,4,5,6,7,8]

[1,2,3,4,5,6,7,8]

In [36]:
map (`mod` 3 == 0) [1..100]

: 

In [37]:
map ((==0) . (`mod` 3)) [1..100]

[False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False]

In [48]:
count :: String -> Int
count xs = length (nub (map toUpper (filter isAlpha xs)))

count' :: String -> Int
count' xs = (length . nub . (map toUpper) . (filter isAlpha)) xs

1

In [50]:
map ((map toLower) . fst) [("Aldi", 100), ("Hyvee", 45), ("Walmart", 40)]

["aldi","hyvee","walmart"]

In [ ]:
getFirst :: [a] -> a
getFirst xs = head xs

getFirst' :: [a] -> a
getFirst' = head

In [51]:
count'' :: String -> Int
count'' = length . nub . (map toUpper) . (filter isAlpha)

In [ ]:
add